# Rule based Predict Price

## load df from s3 origin bucket

In [6]:
from util.s3_manager.manager import S3Manager

date = "201908"

manager = S3Manager(bucket_name="production-bobsim")
origin_df = manager.fetch_objects(key="public_data/open_data_raw_material_price/origin/csv/{filename}.csv".format(
            filename=date
        ))[0]

2020-03-30 12:16:03:manager:<INFO> 1 files is loaded from public_data/open_data_raw_material_price/origin/csv/201908.csv in s3 'production-bobsim'


## subtract one day and one day after

In [7]:
origin_df[origin_df.조사구분명 == "소비자가격"].drop("조사구분명", axis=1).groupby([
            "조사일자", "조사지역명", "조사단위명",
            "표준품목명", "조사가격품목명", "표준품종명", "조사가격품종명"
        ]).mean().reset_index()

,조사일자,조사지역명,조사단위명,표준품목명,조사가격품목명,표준품종명,조사가격품종명,조사구분코드,표준품목코드,조사가격품목코드,조사가격품종코드,조사등급코드,표준단위코드,표준단위명,당일조사가격,전일조사가격,조사지역코드,표준시장코드,조사가격시장코드
0,2019-08-01,강원,100G,감자,감자,수미,수미,7.0,501.0,152.0,1.0,1.0,NaN,NaN,238.0,258.0,3204.0,7024201.0,1.0
1,2019-08-01,강원,100G,건제품,건포도,건포도(수입),수입,7.0,9117.0,426.0,0.0,2.0,NaN,NaN,1450.0,1450.0,3204.0,7024201.0,1.0
2,2019-08-01,강원,100G,고추,붉은고추,홍고추,일반,7.0,1205.0,243.0,0.0,1.0,NaN,NaN,990.0,1050.0,3204.0,7024201.0,1.0
3,2019-08-01,강원,100G,고추,풋고추,꽈리고추,꽈리고추,7.0,1205.0,242.0,2.0,1.0,NaN,NaN,1050.0,1050.0,3204.0,7024201.0,1.0
4,2019-08-01,강원,100G,고추,풋고추,청양,청양고추,7.0,1205.0,242.0,3.0,1.0,NaN,NaN,670.0,1120.0,3204.0,7024201.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40507,2019-08-30,충북청주,500G,팥,팥(적두),붉은팥,국산,7.0,302.0,142.0,0.0,1.0,NaN,NaN,7830.0,7830.0,3300.0,4028704.0,1.0
40508,2019-08-30,충북청주,5KG,천연조미료류,굵은소금,천일염,일반,7.0,9391.0,652.0,0.0,1.0,NaN,NaN,5830.0,5830.0,3300.0,4028704.0,1.0
40509,2019-08-30,충북청주,5마리,꽁치류,꽁치,꽁치(수입),수입냉동,7.0,6114.0,612.0,1.0,2.0,NaN,NaN,3570.0,3570.0,3300.0,4028704.0,1.0
40510,2019-08-30,충북청주,5마리,새우류,새우,흰다리새우,일반,7.0,6304.0,654.0,1.0,2.0,NaN,NaN,5290.0,5290.0,3300.0,4028704.0,1.0
